#AUDIT LOG TABLE

In [0]:
%sql

CREATE OR REPLACE TABLE destination (
  id INT,
  name STRING,
  country STRING,
  active BOOLEAN
)
USING DELTA
LOCATION "dbfs:/FileStore/tables/delta/auditlog/"

In [0]:
%sql

INSERT INTO
  destination (id, name, country, active) 
VALUES
  (1, "Laura", "France", true),
  (2,"Michael","Spain",true),
  (3,"Luis","Germany",false),
  (5,"Liam","France",true);

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql

SELECT * FROM destination;

id,name,country,active
1,Laura,France,true
2,Michael,Spain,true
3,Luis,Germany,false
5,Liam,France,true


In [0]:
data_source = [
  (1,"Laura","USA",True),
  (2,"Michael","Spain",False),
  (3,"Luis","Germany",False),
  (4,"Xi","China",True)
]

schema_source = "id INTEGER, name STRING, country STRING, active BOOLEAN"

In [0]:
df_source = spark.createDataFrame(data=data_source,schema=schema_source)

df_source.show()

+---+-------+-------+------+
| id|   name|country|active|
+---+-------+-------+------+
|  1|  Laura|    USA|  true|
|  2|Michael|  Spain| false|
|  3|   Luis|Germany| false|
|  4|     Xi|  China|  true|
+---+-------+-------+------+



In [0]:
df_source.createOrReplaceTempView("source")

In [0]:
%sql

DELETE FROM destination WHERE id=5;

num_affected_rows
1


In [0]:
%sql

MERGE INTO destination AS d
USING source AS s
  ON d.id = s.id
WHEN MATCHED
  THEN UPDATE SET
    d.name = s.name,
    d.country = s.country,
    d.active = s.active
WHEN NOT MATCHED THEN
  INSERT (id,name,country,active) VALUES (id,name,country,active)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
4,3,0,1


In [0]:
%sql

SELECT * FROM destination

id,name,country,active
2,Michael,Spain,false
3,Luis,Germany,false
1,Laura,USA,true
4,Xi,China,true


##Delta Table History

In [0]:
from delta.tables import *

In [0]:
history_dt = DeltaTable.forPath(spark,"dbfs:/FileStore/tables/delta/auditlog/")

display(history_dt.history(2))

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
7,2024-08-25T16:58:14.000+0000,8904480319608266,andresmunozpampillonaws@gmail.com,MERGE,"Map(predicate -> [""(id#5012 = id#4475)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(3525679404895447),0825-162823-fpajujac,6,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 4, numTargetBytesAdded -> 5082, numTargetBytesRemoved -> 1300, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 3, executionTimeMs -> 6376, numTargetRowsInserted -> 1, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 2527, numTargetRowsUpdated -> 3, numOutputRows -> 4, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 4, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 2845)",null,Databricks-Runtime/12.2.x-scala2.12
6,2024-08-25T16:58:04.000+0000,8904480319608266,andresmunozpampillonaws@gmail.com,DELETE,"Map(predicate -> [""(id#4504 = 5)""])",null,List(3525679404895447),0825-162823-fpajujac,5,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1351, numCopiedRows -> 3, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2119, numDeletedRows -> 1, scanTimeMs -> 905, numAddedFiles -> 1, numAddedBytes -> 1300, rewriteTimeMs -> 1213)",null,Databricks-Runtime/12.2.x-scala2.12


## Explode Operation Metrics

In [0]:
from pyspark.sql.functions import explode, lit, current_timestamp

In [0]:
df_last_operation = history_dt.history(1)

df_explode = df_last_operation.select(df_last_operation.operation, explode(df_last_operation.operationMetrics))

df_explode_select = df_explode.select(df_explode.operation, df_explode.key, df_explode.value.cast('int'))

display(df_explode_select)

operation,key,value
MERGE,numTargetRowsCopied,0
MERGE,numTargetRowsDeleted,0
MERGE,numTargetFilesAdded,4
MERGE,numTargetBytesAdded,5082
MERGE,numTargetBytesRemoved,1300
MERGE,numTargetDeletionVectorsAdded,0
MERGE,numTargetRowsMatchedUpdated,3
MERGE,executionTimeMs,6376
MERGE,numTargetRowsInserted,1
MERGE,numTargetRowsMatchedDeleted,0


## Pivot 

In [0]:
display(df_explode_select.groupBy("operation").pivot("key").sum("value"))

operation,executionTimeMs,numOutputRows,numSourceRows,numTargetBytesAdded,numTargetBytesRemoved,numTargetChangeFilesAdded,numTargetDeletionVectorsAdded,numTargetDeletionVectorsRemoved,numTargetFilesAdded,numTargetFilesRemoved,numTargetRowsCopied,numTargetRowsDeleted,numTargetRowsInserted,numTargetRowsMatchedDeleted,numTargetRowsMatchedUpdated,numTargetRowsNotMatchedBySourceDeleted,numTargetRowsNotMatchedBySourceUpdated,numTargetRowsUpdated,rewriteTimeMs,scanTimeMs
MERGE,6376,4,4,5082,1300,0,0,0,4,1,0,0,1,0,3,0,0,3,2845,2527


In [0]:
df_audit_log = df_explode_select \
    .groupBy("operation").pivot("key").sum("value") \
    .select("operation","numTargetRowsUpdated","numTargetRowsInserted","numTargetRowsDeleted") \
    .withColumn("updated_time",lit(current_timestamp()))

display(df_audit_log)

operation,numTargetRowsUpdated,numTargetRowsInserted,numTargetRowsDeleted,updated_time
MERGE,3,1,0,2024-08-25T17:24:00.097+0000


In [0]:
df_audit_log.createTempView("audit")

## Audit log

In [0]:
%sql
CREATE OR REPLACE TABLE audit_log(
  operation STRING,
  numTargetRowsUpdated INT,
  numTargetRowsInserted INT,
  numTargetRowsDeleted INT,
  updated_time TIMESTAMP
)

In [0]:
%sql
INSERT INTO audit_log
  SELECT * FROM audit;

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
SELECT * FROM audit_log

operation,numTargetRowsUpdated,numTargetRowsInserted,numTargetRowsDeleted,updated_time
MERGE,3,1,0,2024-08-25T17:26:29.356+0000
